### **Cumulative Duke University Pep Band Analysis**

**Importing Libraries**<br>
Prior to running our code, we need to import libraries and our data files.

In [1]:
import pandas as pd
import os

**Set the Start Date**<br>
Write a note that says when songs have started being tracked.

In [2]:
start_date = "02/01/24"

**Importing Data**<br>
Prior to running our code, we'll merge all the song data by combining them all into one file.

In [3]:
# access all the CSV files from the input data folder
INPUT_DIRECTORY = "../data/semester/"
OUTPUT_PATH = "../data/cumulative/"
OUTPUT_FILENAME = 'Duke Band Song Statistics - Cumulative.csv'
dataframes = []

# loop through the CSV files to create pandas dataframes of them
for filename in os.listdir(INPUT_DIRECTORY):
    if filename.endswith(".csv"):
        filepath = os.path.join(INPUT_DIRECTORY, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)
        
# merge the dataframes together and store the cumulative result
merged_df = pd.concat(dataframes, ignore_index=True)

Let's sort the merged data by date.

In [4]:
# sorted_by_name_df = merged_df.sort_values(by=['Opponent', 'Event'], key=find_name)
sorted_by_date_df = merged_df.sort_values(by='Date')
sorted_by_date_df.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILENAME), index=False)

**Song Frequency**<br>
Let's check how many times each song was played since records started in February 2024. First, we need to define a function that can find the song frequency.

In [5]:
"""
find_song_frequency function

Input:  input_path = String filepath of the Pep Band csv file
Output: freqs      = Dictionary that tracks frequencies of all songs played

"""
def find_song_frequency(input_path: str):
    
    # store the frequencies of each song played in a dictionary
    # where the key = song name, and value = frequency
    freqs = dict()
    
    with open(input_path, "r") as input_file:
        next(input_file, None) # skip the header
        
        for line in input_file:
            row = line.split(",")
            
            # for each song, add it to the frequency
            song = row[3]
            if song not in freqs:
                freqs[song] = 1
            else:
                freqs[song] += 1
    
    return freqs

Now that we have defined the function, we can find the song statistics.

In [24]:
INPUT_FILEPATH = "../data/cumulative/Duke Band Song Statistics - Cumulative.csv"
freqs = find_song_frequency(INPUT_FILEPATH)

We can first check how many songs we played in the entirety of February 2024.

In [25]:
TOTAL_COUNT = sum(freqs.values())
print(f"Since {start_date}, the Duke University Pep Band has played at least {TOTAL_COUNT} times.")

Since 02/01/24, the Duke University Pep Band has played at least 1000 times.


After finding the total, let's break down the results by song name. I sort the results based in decreasing frequency.

In [26]:
# set a buffer for how many long all the "song_name" entries should be
BUFFER = 30

In [27]:
print(f"Since {start_date}, the Duke University Pep Band has played:")
for song_name in sorted(freqs, key=lambda x: -freqs[x]):
    print(f"{song_name.ljust(BUFFER)} {freqs[song_name]} times")

Since 02/01/24, the Duke University Pep Band has played:
Blue & White                   124 times
Fight Fight (Intro)            103 times
Devil                          80 times
Fight Fight                    67 times
Zing It!                       52 times
Can't                          49 times
Everytime                      32 times
Dear Old Duke                  32 times
The Whip                       31 times
That's What I Want             29 times
Pumpkin                        28 times
Dance the Night                26 times
Runaway Baby                   25 times
Uma Thurman                    22 times
Overpass                       19 times
Sail                           18 times
Mortal                         15 times
That That                      15 times
Spell                          13 times
Wipeout                        13 times
Love from the Other Side       12 times
HOT TO GO!                     12 times
Lucky Strike                   11 times
Uptown Funk          

**FlipFolder Frequency**<br>
Let's see what percentage of songs in the FlipFolder were played in February 2024. Note that "D. Ditty", "t.gogo", "Pumpkin", and "Fight Fight (Intro)" are not considered FlipFolder songs, as they do not have FlipFolder entries. We also have to ignore instances where we play fight songs for the away team. For the 2024-25 Football Season, these songs are "Elon University Fight Song," "UConn Husky", "FSU Fight Song".

In [28]:
TOTAL_FLIPFOLDER_SONGS = 100
NON_FLIPFOLDER_SONGS = 4
AWAY_FIGHT_SONGS = 3

In [29]:
ACTIVE_SONGS = len(freqs.keys())-NON_FLIPFOLDER_SONGS-AWAY_FIGHT_SONGS
print(f"Since {start_date}, {ACTIVE_SONGS}/{TOTAL_FLIPFOLDER_SONGS} songs in the FlipFolder have been played.")

Since 02/01/24, 58/100 songs in the FlipFolder have been played.


**Fight vs Non-Fight Songs**<br>
What percentage of the songs played were fight songs? The Duke University Pep Band has the following fight songs:

- Fight Fight
- Blue & White
- Zing It!
- Devil
- Can't
- Dear Old Duke


We can identify fight songs using a set.

In [30]:
fight_songs = {"Fight Fight", "Fight Fight (Intro)", "Blue & White", "Zing It!",
               "Devil", "Can't", "Dear Old Duke"}

We can define a function to count the number of fight songs.

In [31]:
"""
count_fight_songs function

Input:  freqs       = Dictionary that tracks frequencies of all songs played
        fight_songs = Set of fight song string names
Output: count       = Integer representing fight song playing count

"""
def count_fight_songs(freqs: dict, fight_songs: set):
    
    # store the number of times that fights songs have been played
    count = 0
    
    # go through each song in the frequency dict
    for song_name in freqs:
        if song_name in fight_songs:
            count += freqs[song_name]
    
    # return the count once calculated
    return count

We can now check the number of times that fight songs were played.

In [32]:
FIGHT_SONG_COUNT = count_fight_songs(freqs, fight_songs)
TOTAL_COUNT = sum(freqs.values())

print(f"Since {start_date}, {FIGHT_SONG_COUNT}/{TOTAL_COUNT} songs that were played were fight songs.")

Since 02/01/24, 507/1000 songs that were played were fight songs.


From this, we can conclude that these 6 songs make up a whopping 308/581 (~53.01%) of the pieces played!

**Songs by Date**<br>
On what day was the least number of songs played? On what day was the most played?

In [33]:
"""
find_number_of_songs_by_date function

Input:  input_path = String filepath of the Pep Band csv file
Output: freqs      = Dictionary that tracks number of songs played by date

"""
def find_number_of_songs_by_date(input_path: str):
    
    # store the number of songs played by date
    # where the key = game date, and value = number of songs played
    dates = dict()
    
    # store the current date of records
    curr_date = ""
    
    with open(input_path, "r") as input_file:
        next(input_file, None) # skip the header
        
        for line in input_file:
            row = line.split(",")
            
            # update dates if the value isn't null
            curr_date = row[0]
            if len(row[9]) == 1:
                curr_date = " ".join(row[0:3])
            else:
                curr_date = " ".join([row[0], row[9][:-1]])
            
            # increase the number of songs played at that date
            if curr_date not in dates:
                dates[curr_date] = 1
            else:
                dates[curr_date] += 1
    
    return dates

First, find the number of songs played on each date. Then, output the results by printing them.

In [34]:
dates = find_number_of_songs_by_date(INPUT_FILEPATH)

In [35]:
BUFFER = 40
print(f"Since {start_date}, the Duke University Pep Band has played at these events:")
for curr_date in sorted(dates, key=lambda x: -dates[x]):
    print(f"{curr_date.ljust(BUFFER)} {dates[curr_date]} times")

Since 02/01/24, the Duke University Pep Band has played at these events:
09/14/24 UConn FB                        49 times
10/26/24 SMU FB                          46 times
04/20/24 Duke FB                         45 times
09/28/24 UNC FB                          45 times
01/02/25 Ole Miss FB                     43 times
11/23/24 Virginia Tech FB                43 times
10/18/24 Florida State FB                41 times
08/30/24 Elon FB                         38 times
12/04/24 Auburn MBB                      38 times
03/09/24 UNC MBB                         36 times
10/19/24 Lincoln (Pa.) MBB               34 times
11/16/24 Wofford MBB                     32 times
02/07/24 Notre Dame MBB                  31 times
02/10/24 Boston College MBB              30 times
02/28/24 Louisville MBB                  30 times
11/04/24 Maine MBB                       29 times
12/10/24 Incarnate Word MBB              29 times
10/04/24 Countdown to Craziness          27 times
02/19/24 Notre Dame WBB    

In [36]:
total_songs_played = sum(dates.values())
number_of_events = len(dates.keys())
avg_songs_per_game = total_songs_played / number_of_events

print(f"Since {start_date}, over a total of {number_of_events} events, an average of {avg_songs_per_game:.2f} songs were played per event.")


Since 02/01/24, over a total of 42 events, an average of 23.81 songs were played per event.


**Early Stops**<br>
Let's count the number of songs stopped early.

In [37]:
"""
find_early_stops function

Input:  input_path  = String filepath of the Pep Band csv file
Output: early_stops = Dictionary that tracks frequencies of songs stopped early

"""
def find_early_stops(input_path: str):
    
    # store the frequencies of each song stopped early in a dictionary
    # where the key = song name, and value = early stop frequency
    early_stops = dict()
    
    with open(input_path, "r") as input_file:
        next(input_file, None) # skip the header
        
        for line in input_file:
            row = line.split(",")
            
            # only track songs stopped early
            if row[7] != "Y":
                continue
            
            # update the frequency for early songs
            song = row[3]
            if song not in early_stops:
                early_stops[song] = 1
            else:
                early_stops[song] += 1
    
    return early_stops

In [38]:
early_stops = find_early_stops(INPUT_FILEPATH)
print(f"Since {start_date}, the Duke University Pep Band has stopped early for these pieces:")
for song_name in sorted(early_stops, key=lambda x: -early_stops[x]):
    print(f"{song_name.ljust(BUFFER)} {early_stops[song_name]} times")

Since 02/01/24, the Duke University Pep Band has stopped early for these pieces:


**Periods**<br>
Let's count the number of songs played during each period of the game.

In [39]:
"""
find_by_period function

Input:  input_path = String filepath of the Pep Band csv file
Output: periods    = Dictionary that tracks songs by period played

"""
def find_by_period(input_path: str):
    
    # store the frequencies of each song played for each period in a dictionary
    # where the key = song name, and value = period frequency
    periods = dict()
    
    with open(input_path, "r") as input_file:
        next(input_file, None) # skip the header
        
        for line in input_file:
            row = line.split(",")
            
            """
            NOTE: I'm only filtering for football games here.
            Remove this `if` statement if you don't want to do this.
            """
            if row[2] != "FB":
                continue
            
            # update the frequency for songs by period
            p = row[4]
            if p not in periods:
                periods[p] = 1
            else:
                periods[p] += 1
    
    return periods

In [40]:
periods = find_by_period(INPUT_FILEPATH)
print(f"Since {start_date}, the Duke University Pep Band has played by period:")
for song_name in sorted(periods, key=lambda x: -periods[x]):
    print(f"{song_name.ljust(BUFFER)} {periods[song_name]} times")

Since 02/01/24, the Duke University Pep Band has played by period:
4Q                                       92 times
Pregame                                  67 times
3Q                                       48 times
2Q                                       46 times
1Q                                       37 times
Halftime                                 26 times
Devil Walk                               20 times
Postgame                                 13 times
1OT                                      1 times
